Submission for Subhojit Halder

# TASK 1

## Quantum Computing Algorithm

This Python code helps find smaller numbers than the integer 'k' in the array 'list_n.' Firstly, all the values that we use are classical bit values. In this code, we first convert the classical bits into binary bits using the function **'convert_to_binary_with_padding()'**. This function converts all the numbers to binary values with the same length by applying padding to the bits.
The critical thing to note is that storing the values in the quantum circuit takes little endian encoding scheme. However, while reading it, the quantum circuit reads the values in a big-endian encoding scheme, so to manage this, the command **'flip_strings_in_array()'** is used to take care of the encoding scheme.

**"binary_converted"** contains the actual array of encoded values in this encoded values

Now, coming to the actual coding part. A quantum comparator is used where a register stores the input value **'k'** and the function **' IntegerComparator(num_state_qubits=len(binary_converted[0]), value=k, geq=True)'** is used. In this -
**'num_state_qubits'** = is the number of qubits that will hold the integer and will return a comparator of size twice this value
**'value'** = the registered stored value from which the comparison is made; hence **'k'**
**'geq'** = If set to true it will return **"1" if the to be compared value is >= k** and will return **"0" if the to be compared value is < k**
So, using this, a comparator is created. A comparator does one operation at a time, so it can only compare 1 unit with the value "k."

The measurement bit **creg = 1** is used so that the output is either 0 or 1.

A for loop is created where it takes the individual element of the encoded array binary_converted and creates Pauli x gates if the bitplace is "1," and then a comparator is added. The values are measured at the last bit where the gate exists, so if the last bit is at q[3], it will measure the output at q[4]. Then, a barrier is added for uniformity, and all the bits using the measurement bit are set to 0 again using the **".reset()"** function so that the previous results do not affect the calculations of the following bits.

The final count values are stored into **"counts_aray"** as 0 or 1 following the above conditions.
The final outputs are stored in **"fin_output"** by retrieving the indices for which counts_aray is **0**.

**"fin_output"** is the return statement used for the function **"less_than_k()"**.

4 Corresponding Outputs **"A," "B," "C," and "D"** are Shown to prove the validity of the code and whether it is working or not.

In [1]:
from qiskit import QuantumRegister, ClassicalRegister
from qiskit import QuantumCircuit, execute
from qiskit.circuit.library import IntegerComparator
from qiskit import Aer

def convert_to_binary_with_padding(arr):
    max_num = max(arr)
    max_length = len(bin(max_num)) - 2
    binary_arr = []
    for num in arr:
        binary_num = bin(num)[2:]
        padded_binary_num = binary_num.zfill(max_length)
        binary_arr.append(padded_binary_num)
    return binary_arr

def flip_strings_in_array(string_array):
    flipped_array = [string[::-1] for string in string_array]
    return flipped_array

In [2]:
def  less_than_k(k , list_n):
    backend = Aer.get_backend('aer_simulator')
    
    binary_converted = flip_strings_in_array(convert_to_binary_with_padding(list_n)) #convert_to_binary_with_padding(list_n) 
    counts_aray = []
    fin_output = []
    
    q1 = QuantumRegister(2*len(binary_converted[0]),'q')
    c1 = ClassicalRegister(1,'c')
    circuit1 = QuantumCircuit(q1,c1)
    
    for i in range(len(binary_converted)):
        comparator_i = IntegerComparator(num_state_qubits=len(binary_converted[0]), value=k, geq=True)
        for j in range(len(binary_converted[i])):
            if binary_converted[i][j] == '1':
                circuit1.x(q1[j])
        circuit1.compose(comparator_i, inplace=True)
        circuit1.measure(q1[len(binary_converted[0])], c1[0])
        job = execute(circuit1, backend, shots=1)
        counts = job.result().get_counts()
        counts_aray.append(int(str(counts)[2]))
        circuit1.barrier(q1)
        for j in range(len(binary_converted[0])+1):
            circuit1.reset(q1[j])
        
    print(circuit1)
    for i in range(len(counts_aray)):
        if counts_aray[i] == 0:
            fin_output.append(list_n[i])
    
    return fin_output

In [3]:
A = less_than_k(7,[4,9,11,14,1,13,6,15])
print(A)

          ┌──────┐    ░      ┌───┐┌──────┐    ░      ┌───┐┌──────┐    ░      »
q_0: ─────┤0     ├────░──|0>─┤ X ├┤0     ├────░──|0>─┤ X ├┤0     ├────░──|0>─»
          │      │    ░      └───┘│      │    ░      ├───┤│      │    ░      »
q_1: ─────┤1     ├────░──|0>──────┤1     ├────░──|0>─┤ X ├┤1     ├────░──|0>─»
     ┌───┐│      │    ░           │      │    ░      └───┘│      │    ░      »
q_2: ┤ X ├┤2     ├────░──|0>──────┤2     ├────░──|0>──────┤2     ├────░──|0>─»
     └───┘│      │    ░      ┌───┐│      │    ░      ┌───┐│      │    ░      »
q_3: ─────┤3     ├────░──|0>─┤ X ├┤3     ├────░──|0>─┤ X ├┤3     ├────░──|0>─»
          │  cmp │┌─┐ ░      └───┘│  cmp │┌─┐ ░      └───┘│  cmp │┌─┐ ░      »
q_4: ─────┤4     ├┤M├─░──|0>──────┤4     ├┤M├─░──|0>──────┤4     ├┤M├─░──|0>─»
          │      │└╥┘ ░           │      │└╥┘ ░           │      │└╥┘ ░      »
q_5: ─────┤5     ├─╫──░───────────┤5     ├─╫──░───────────┤5     ├─╫──░──────»
          │      │ ║  ░           │      │ ║  ░     

In [4]:
B = less_than_k(4,[1,2,10,0,4])
print(B)

     ┌───┐┌──────┐    ░           ┌──────┐    ░           ┌──────┐    ░      »
q_0: ┤ X ├┤0     ├────░──|0>──────┤0     ├────░──|0>──────┤0     ├────░──|0>─»
     └───┘│      │    ░      ┌───┐│      │    ░      ┌───┐│      │    ░      »
q_1: ─────┤1     ├────░──|0>─┤ X ├┤1     ├────░──|0>─┤ X ├┤1     ├────░──|0>─»
          │      │    ░      └───┘│      │    ░      └───┘│      │    ░      »
q_2: ─────┤2     ├────░──|0>──────┤2     ├────░──|0>──────┤2     ├────░──|0>─»
          │      │    ░           │      │    ░      ┌───┐│      │    ░      »
q_3: ─────┤3     ├────░──|0>──────┤3     ├────░──|0>─┤ X ├┤3     ├────░──|0>─»
          │  cmp │┌─┐ ░           │  cmp │┌─┐ ░      └───┘│  cmp │┌─┐ ░      »
q_4: ─────┤4     ├┤M├─░──|0>──────┤4     ├┤M├─░──|0>──────┤4     ├┤M├─░──|0>─»
          │      │└╥┘ ░           │      │└╥┘ ░           │      │└╥┘ ░      »
q_5: ─────┤5     ├─╫──░───────────┤5     ├─╫──░───────────┤5     ├─╫──░──────»
          │      │ ║  ░           │      │ ║  ░     

In [5]:
C = less_than_k(10,[16,12,11])
print(C)

          ┌──────┐    ░           ┌──────┐    ░      ┌───┐┌──────┐    ░      
q_0: ─────┤0     ├────░──|0>──────┤0     ├────░──|0>─┤ X ├┤0     ├────░──|0>─
          │      │    ░           │      │    ░      ├───┤│      │    ░      
q_1: ─────┤1     ├────░──|0>──────┤1     ├────░──|0>─┤ X ├┤1     ├────░──|0>─
          │      │    ░      ┌───┐│      │    ░      └───┘│      │    ░      
q_2: ─────┤2     ├────░──|0>─┤ X ├┤2     ├────░──|0>──────┤2     ├────░──|0>─
          │      │    ░      ├───┤│      │    ░      ┌───┐│      │    ░      
q_3: ─────┤3     ├────░──|0>─┤ X ├┤3     ├────░──|0>─┤ X ├┤3     ├────░──|0>─
     ┌───┐│      │    ░      └───┘│      │    ░      └───┘│      │    ░      
q_4: ┤ X ├┤4     ├────░──|0>──────┤4     ├────░──|0>──────┤4     ├────░──|0>─
     └───┘│  cmp │┌─┐ ░           │  cmp │┌─┐ ░           │  cmp │┌─┐ ░      
q_5: ─────┤5     ├┤M├─░──|0>──────┤5     ├┤M├─░──|0>──────┤5     ├┤M├─░──|0>─
          │      │└╥┘ ░           │      │└╥┘ ░           │     

In [6]:
D = less_than_k(10,[4,9,8,6])
print(D)

          ┌──────┐    ░      ┌───┐┌──────┐    ░           ┌──────┐    ░      »
q_0: ─────┤0     ├────░──|0>─┤ X ├┤0     ├────░──|0>──────┤0     ├────░──|0>─»
          │      │    ░      └───┘│      │    ░           │      │    ░      »
q_1: ─────┤1     ├────░──|0>──────┤1     ├────░──|0>──────┤1     ├────░──|0>─»
     ┌───┐│      │    ░           │      │    ░           │      │    ░      »
q_2: ┤ X ├┤2     ├────░──|0>──────┤2     ├────░──|0>──────┤2     ├────░──|0>─»
     └───┘│      │    ░      ┌───┐│      │    ░      ┌───┐│      │    ░      »
q_3: ─────┤3     ├────░──|0>─┤ X ├┤3     ├────░──|0>─┤ X ├┤3     ├────░──|0>─»
          │  cmp │┌─┐ ░      └───┘│  cmp │┌─┐ ░      └───┘│  cmp │┌─┐ ░      »
q_4: ─────┤4     ├┤M├─░──|0>──────┤4     ├┤M├─░──|0>──────┤4     ├┤M├─░──|0>─»
          │      │└╥┘ ░           │      │└╥┘ ░           │      │└╥┘ ░      »
q_5: ─────┤5     ├─╫──░───────────┤5     ├─╫──░───────────┤5     ├─╫──░──────»
          │      │ ║  ░           │      │ ║  ░     